In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeClassifier
import numpy as np

def decision_tree_pipeline(
    df,                      # Input DataFrame
    target_column,           # Name of target column
    numeric_features,        # List of numeric feature column names
    categorical_features,    # List of categorical feature column names
    test_size=0.2,           # Test set size
    max_depth=None,          # Optional: max depth of the tree
    custom_input=None        # Optional custom input for prediction (as DataFrame)
):
    # Split features and target
    X = df.drop(target_column, axis=1)
    y = df[target_column]

    # Define preprocessing
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', StandardScaler(), numeric_features),
            ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
        ])

    # Define pipeline
    model = Pipeline(steps=[
        ('preprocess', preprocessor),
        ('classifier', DecisionTreeClassifier(criterion='entropy', max_depth=max_depth, random_state=42))
    ])

    # Train-test split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=42)

    # Train model
    model.fit(X_train, y_train)

    # Predict on test set
    test_prediction = model.predict(X_test)

    print("Test predictions:\n", test_prediction)

    # Custom input prediction (if provided)
    if custom_input is not None:
        custom_prediction = model.predict(custom_input)
        custom_proba = model.predict_proba(custom_input)

        print("\nCustom input:")
        print(custom_input)
        print("Predicted class:", custom_prediction)
        print("Class probabilities:", custom_proba)

    return model  # Return model for reuse

# ------------------ Example usage ------------------

# Sample dataset
df = pd.read_csv('play_tennis.csv')
df = df.drop('day', axis=1)

# Example custom input
custom_input = pd.DataFrame([{
    'outlook': 'Sunny',
    'temp': 'Hot',
    'humidity': 'High',
    'wind': 'Weak'
}])

# Run the pipeline
decision_tree_pipeline(
    df=df,
    target_column='play',
    numeric_features=[],
    categorical_features=['outlook', 'temp', 'humidity','wind'],
    test_size=0.4,
    max_depth=3,
    custom_input=custom_input
)


Test predictions:
 ['Yes' 'No' 'No' 'Yes' 'Yes' 'Yes']

Custom input:
  outlook temp humidity  wind
0   Sunny  Hot     High  Weak
Predicted class: ['No']
Class probabilities: [[1. 0.]]


Pipeline(steps=[('preprocess',
                 ColumnTransformer(transformers=[('num', StandardScaler(), []),
                                                 ('cat',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['outlook', 'temp',
                                                   'humidity', 'wind'])])),
                ('classifier',
                 DecisionTreeClassifier(criterion='entropy', max_depth=3,
                                        random_state=42))])

In [3]:
import pandas as pd
import numpy as np
import math

# ---------------------- Helper Functions ----------------------

def entropy(y):
    values, counts = np.unique(y, return_counts=True)
    probs = counts / counts.sum()
    return -np.sum(probs * np.log2(probs))

def information_gain(df, feature, target):
    total_entropy = entropy(df[target])
    values, counts = np.unique(df[feature], return_counts=True)
    weighted_entropy = sum(
        (counts[i] / np.sum(counts)) * entropy(df[df[feature] == values[i]][target])
        for i in range(len(values))
    )
    return total_entropy - weighted_entropy

def best_split(df, features, target):
    gains = {feature: information_gain(df, feature, target) for feature in features}
    return max(gains, key=gains.get)

def majority_class(y):
    return y.value_counts().idxmax()

# ---------------------- ID3 Core ----------------------

class ID3Node:
    def __init__(self, feature=None, children=None, is_leaf=False, prediction=None):
        self.feature = feature
        self.children = children if children else {}
        self.is_leaf = is_leaf
        self.prediction = prediction

class ID3Classifier:
    def __init__(self, max_depth=None):
        self.max_depth = max_depth
        self.tree = None

    def fit(self, df, target_column):
        features = [col for col in df.columns if col != target_column]
        self.tree = self._build_tree(df, features, target_column, depth=0)

    def _build_tree(self, df, features, target_column, depth):
        y = df[target_column]

        # Stopping conditions
        if len(np.unique(y)) == 1:
            return ID3Node(is_leaf=True, prediction=y.iloc[0])
        if len(features) == 0 or (self.max_depth is not None and depth >= self.max_depth):
            return ID3Node(is_leaf=True, prediction=majority_class(y))

        best_feature = best_split(df, features, target_column)
        node = ID3Node(feature=best_feature)

        for value in df[best_feature].unique():
            subset = df[df[best_feature] == value]
            if subset.empty:
                child = ID3Node(is_leaf=True, prediction=majority_class(y))
            else:
                new_features = [f for f in features if f != best_feature]
                child = self._build_tree(subset, new_features, target_column, depth + 1)
            node.children[value] = child

        return node

    def _predict_instance(self, node, instance):
        if node.is_leaf:
            return node.prediction
        feature_value = instance[node.feature]
        if feature_value in node.children:
            return self._predict_instance(node.children[feature_value], instance)
        else:
            return None  # unknown value

    def predict(self, df):
        return df.apply(lambda x: self._predict_instance(self.tree, x), axis=1)

# ---------------------- Example Usage ----------------------

# Sample dataset (same as before)
df = pd.read_csv('play_tennis.csv')
df = df.drop('day', axis=1)

# Train
model = ID3Classifier(max_depth=3)
model.fit(df, target_column='play')

# Predict on custom input
custom_input = pd.DataFrame([{
    'outlook': 'Sunny',
    'temp': 'Hot',
    'humidity': 'High',
    'wind': 'Weak'
}])

predictions = model.predict(custom_input)

print("Custom input prediction:", predictions.tolist())


Custom input prediction: ['No']


In [5]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeClassifier
import numpy as np

def decision_tree_pipeline(
    df,                      # Input DataFrame
    target_column,           # Name of target column
    numeric_features,        # List of numeric feature column names
    categorical_features,    # List of categorical feature column names
    test_size=0.2,           # Test set size
    max_depth=None,          # Optional: max depth of the tree
    custom_input=None        # Optional custom input for prediction (as DataFrame)
):
    # Split features and target
    X = df.drop(target_column, axis=1)
    y = df[target_column]

    # Define preprocessing
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', StandardScaler(), numeric_features),
            ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
        ])

    # Define pipeline
    model = Pipeline(steps=[
        ('preprocess', preprocessor),
        ('classifier', DecisionTreeClassifier(criterion='entropy', max_depth=max_depth, random_state=42))
    ])

    # Train-test split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=42)

    # Train model
    model.fit(X_train, y_train)

    # Predict on test set
    test_prediction = model.predict(X_test)

    print("Test predictions:\n", test_prediction)

    # Custom input prediction (if provided)
    if custom_input is not None:
        custom_prediction = model.predict(custom_input)
        custom_proba = model.predict_proba(custom_input)

        print("\nCustom input:")
        print(custom_input)
        print("Predicted class:", custom_prediction)
        print("Class probabilities:", custom_proba)

    return model  # Return model for reuse

# ------------------ Example usage ------------------

# Sample dataset
df = pd.read_csv('Iris.csv')
df = df.drop('Id', axis=1)


# Example custom input
custom_input = pd.DataFrame([{
    'SepalLengthCm': 5.1,
    'SepalWidthCm': 3.5,
    'PetalLengthCm': 1.4,
    'PetalWidthCm': 0.2
}])

# Run the pipeline
decision_tree_pipeline(
    df=df,
    target_column='Species',
    numeric_features=['SepalLengthCm', 'SepalWidthCm', 'PetalLengthCm', 'PetalWidthCm'],
    categorical_features=[],
    test_size=0.4,
    max_depth=3,
    custom_input=custom_input
)


Test predictions:
 ['Iris-versicolor' 'Iris-setosa' 'Iris-virginica' 'Iris-versicolor'
 'Iris-versicolor' 'Iris-setosa' 'Iris-versicolor' 'Iris-virginica'
 'Iris-versicolor' 'Iris-versicolor' 'Iris-virginica' 'Iris-setosa'
 'Iris-setosa' 'Iris-setosa' 'Iris-setosa' 'Iris-versicolor'
 'Iris-virginica' 'Iris-versicolor' 'Iris-versicolor' 'Iris-virginica'
 'Iris-setosa' 'Iris-virginica' 'Iris-setosa' 'Iris-virginica'
 'Iris-virginica' 'Iris-virginica' 'Iris-virginica' 'Iris-virginica'
 'Iris-setosa' 'Iris-setosa' 'Iris-setosa' 'Iris-setosa' 'Iris-versicolor'
 'Iris-setosa' 'Iris-setosa' 'Iris-virginica' 'Iris-versicolor'
 'Iris-setosa' 'Iris-setosa' 'Iris-setosa' 'Iris-virginica'
 'Iris-versicolor' 'Iris-versicolor' 'Iris-setosa' 'Iris-setosa'
 'Iris-versicolor' 'Iris-versicolor' 'Iris-virginica' 'Iris-versicolor'
 'Iris-virginica' 'Iris-versicolor' 'Iris-virginica' 'Iris-versicolor'
 'Iris-setosa' 'Iris-virginica' 'Iris-versicolor' 'Iris-setosa'
 'Iris-setosa' 'Iris-setosa' 'Iris-versico

Pipeline(steps=[('preprocess',
                 ColumnTransformer(transformers=[('num', StandardScaler(),
                                                  ['SepalLengthCm',
                                                   'SepalWidthCm',
                                                   'PetalLengthCm',
                                                   'PetalWidthCm']),
                                                 ('cat',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  [])])),
                ('classifier',
                 DecisionTreeClassifier(criterion='entropy', max_depth=3,
                                        random_state=42))])

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,1,5.1,3.5,1.4,0.2,Iris-setosa
1,2,4.9,3.0,1.4,0.2,Iris-setosa
2,3,4.7,3.2,1.3,0.2,Iris-setosa
3,4,4.6,3.1,1.5,0.2,Iris-setosa
4,5,5.0,3.6,1.4,0.2,Iris-setosa
